# Validation Dataset Notebook
## Customised to use VoxelNet trained Graphs on custom datasets.
## Please use the csvToKITTI utility tool to convert VeloView CSVs to a compatible output. 

In [ ]:
#!/usr/bin/env python
# -*- coding:UTF-8 -*-

# from comet_ml import Experiment
import glob
import argparse
import os
import time
import tensorflow as tf
from model import RPN3D
from config import cfg
from utils import *
from utils.kitti_loader import iterate_data, sample_test_data,GPULogging
import subprocess


  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/pool.py", line 108, in worke

In [ ]:
# ctx = 'non_urban' 
tag = 'default_pedestrian'
data_dir='/mnt/storage/home/ja17618/scratch/'
output_path = data_dir+'/out/' 
bs=1
vis=True
GPU_AVAILABLE='0'
GPU_USE_COUNT=1


In [ ]:
import cv2
import numpy as np
import os
import sys
import glob
import math
import multiprocessing


from config import cfg
from utils.data_aug import aug_data
from utils.preprocess import process_pointcloud

class Processor:
    def __init__(self, data_tag, f_lidar, data_dir, aug, is_testset):
        self.data_tag=data_tag
        self.f_lidar = f_lidar
        self.data_dir = data_dir
        self.aug = aug
        self.is_testset = is_testset
    
    def __call__(self,load_index):
        if self.aug:
            ret = aug_data(self.data_tag[load_index], self.data_dir)
        else:
            raw_lidar = np.fromfile(self.f_lidar[load_index], dtype=np.float32).reshape((-1, 4))
            if not self.is_testset:
                labels = [line for line in open(self.f_label[load_index], 'r').readlines()]
            else:
                labels = ['']
            tag = self.data_tag[load_index]
            voxel = process_pointcloud(raw_lidar)
            ret = [tag, raw_lidar, voxel, labels]
        return ret

# global pool
TRAIN_POOL = multiprocessing.Pool(4)
VAL_POOL = multiprocessing.Pool(2)

def iterate_val_data(data_dir, shuffle=False, aug=False, is_testset=True, batch_size=1, multi_gpu_sum=1):
    f_lidar = glob.glob(os.path.join(data_dir, 'output', '*.bin'))
    f_lidar.sort()
    
    data_tag = [name.split('/')[-1].split('.')[-2] for name in f_lidar]
      
    nums = len(f_lidar)
    indices = list(range(nums))
    if shuffle:
        np.random.shuffle(indices)

    num_batches = int(math.floor( nums / float(batch_size) ))

    proc=Processor(data_tag, f_lidar, data_dir, aug, is_testset)

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        excerpt = indices[start_idx:start_idx + batch_size]
        
        rets=TRAIN_POOL.map(proc,excerpt)

        tag = [ ret[0] for ret in rets ]
        raw_lidar = [ ret[1] for ret in rets ]
        voxel = [ ret[2] for ret in rets ]
        labels = [ ret[3] for ret in rets ]

        # only for voxel -> [gpu, k_single_batch, ...]
        vox_feature, vox_number, vox_coordinate = [], [], []
        single_batch_size = int(batch_size / multi_gpu_sum)
        for idx in range(multi_gpu_sum):
            _, per_vox_feature, per_vox_number, per_vox_coordinate = build_input(voxel[idx * single_batch_size:(idx + 1) * single_batch_size])
            vox_feature.append(per_vox_feature)
            vox_number.append(per_vox_number)
            vox_coordinate.append(per_vox_coordinate)

        ret = (
               np.array(tag),
               np.array(labels),
               np.array(vox_feature),
               np.array(vox_number),
               np.array(vox_coordinate),
               np.array(raw_lidar)
               )

        yield ret


  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/mnt/storag

In [4]:
# log = open('/mnt/storage/home/ja17618/vox_nurbgpu3.out','w+')
# log.flush()

dataset_dir =data_dir
# val_dir = os.path.join(cfg.CONTEXT_DIR, ctx)
save_model_dir = os.path.join('./save_model', tag)
total_inference_time = []

# experiment = Experiment(api_key="xXtJguCo8yFdU7dpjEpo6YbHw",project_name=exp)
# create output folder

os.makedirs(output_path, exist_ok=True)
os.makedirs(os.path.join(output_path, 'data'), exist_ok=True)

if vis:
    os.makedirs(os.path.join(output_path, 'vis'), exist_ok=True)
# process = subprocess.Popen('nvidia-smi -i 1 --format=csv -l 1 --query-gpu=index,timestamp,power.draw,utilization.gpu,clocks.current.sm,temperature.gpu,memory.used',stdout = log, shell=True)

with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=cfg.GPU_MEMORY_FRACTION,
                        visible_device_list=GPU_AVAILABLE,
                        allow_growth=True)

    config = tf.ConfigProto(
        gpu_options=gpu_options,
        device_count={
            "GPU": GPU_USE_COUNT,
        },
        allow_soft_placement=True,
        log_device_placement=False
    )
    run_metadata = tf.RunMetadata()
    with tf.Session(config=config) as sess:
        model = RPN3D(
            cls=cfg.DETECT_OBJ,
            single_batch_size=bs,
            avail_gpus=GPU_AVAILABLE
        )
        if tf.train.get_checkpoint_state(save_model_dir):
            print("Reading model parameters from %s" % save_model_dir)
            model.saver.restore(
                sess, tf.train.latest_checkpoint(save_model_dir))
        counter=0
#         with experiment.test():
        for batch in iterate_val_data(dataset_dir, shuffle=False, aug=False, is_testset=True, batch_size=bs * GPU_USE_COUNT, multi_gpu_sum=GPU_USE_COUNT):
#             experiment.log_metric("counter",counter)
            tf.profiler.profile(
                options=tf.profiler.ProfileOptionBuilder.float_operation(),run_meta=run_metadata)
            if vis:
                tags, results, bird_views, heatmaps = model.predict_velo_step(sess, batch, vis=True)
            else:
                inference_start_time=time.time()
                tags, results = model.predict_velo_step(sess, batch, vis=False)
                inference_time = time.time()-inference_start_time
                total_inference_time.append(inference_time)

            # ret: A, B
            # A: (N) tag
            # B: (N, N') (class, x, y, z, h, w, l, rz, score)
            for tag, result in zip(tags, results):
                of_path = os.path.join(output_path, 'data', tag + '.txt')
                with open(of_path, 'w+') as f:
                    labels = box3d_to_label([result[:, 1:8]], [result[:, 0]], [result[:, -1]], coordinate='lidar')[0]
                    for line in labels:
                        f.write(line)
                    if len(labels)>0:
                        print('write out {} objects to {}'.format(len(labels), tag))
            # dump visualizations
            if vis:
                for tag, bird_view, heatmap in zip(tags, bird_views, heatmaps):
#                     front_img_path = os.path.join( output_path, 'vis', tag + '_front.jpg'  )
                    bird_view_path = os.path.join( output_path, 'vis', tag + '_bv.jpg'  )
                    heatmap_path = os.path.join( output_path, 'vis', tag + '_heatmap.jpg'  )
#                     cv2.imwrite( front_img_path, front_image )
                    print(bird_view_path)
                    cv2.imwrite( bird_view_path, bird_view )
                    cv2.imwrite( heatmap_path, heatmap )
            counter +=1
# process.kill()
# log.flush()
# log.close()
print('done')

In [ ]:
mean_inference = np.mean(total_inference_time)
min_inference = np.min(total_inference_time)
max_inference =np.max(total_inference_time)
var_inference = np.var(total_inference_time)

print('Min: %fs Max: %fs Mean: %fs Var: %fs ' %( min_inference,max_inference,mean_inference,var_inference))